In [2]:
import consul
import sys
import uuid

In [32]:
def register_service(service_name, service_ip, service_port, consul_ip, consul_port):
    consul_client = consul.Consul(host=consul_ip, port=consul_port)

    service_id = f"{service_name}-{str(uuid.uuid4())[:4]}"

    consul_client.agent.service.register(
    name=service_name,
    service_id=service_id,
    address=service_ip,
    port=service_port,
    check=consul.Check.http(
        url=f"http://{service_ip}:{consul_ip}/health",
        interval="10s",
        timeout="1s",
        deregister="10m"
    )
)

In [3]:
SERVICE_NAME = sys.argv[0][:-3]
SERVICE_ID = f"{SERVICE_NAME}-{str(uuid.uuid4())[:4]}"

CONSUL_IP = "localhost"
CONSUL_PORT = 8500

SERVICE_IP = "host.docker.internal"
SERVICE_PORT = 5000

In [4]:
c = consul.Consul(host=CONSUL_IP, port=CONSUL_PORT)
try:
    leader = c.status.leader()
    print(f"Consul is available. Leader: {leader}")
except Exception as e:
    print(f"Failed to connect to Consul: {e}")

Consul is available. Leader: 127.0.0.1:8300


In [33]:
register_service(SERVICE_NAME, SERVICE_IP, SERVICE_PORT, CONSUL_IP, CONSUL_PORT)

ConnectionError: HTTPConnectionPool(host='localhost', port=8500): Max retries exceeded with url: /v1/agent/service/register (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x75e9fc523a90>: Failed to establish a new connection: [Errno 111] Connection refused'))

### Get all services

In [29]:
services = c.agent.services()
for service_id, service_info in services.items():
    print(f"Service ID: {service_id}")
    print(f"Service Name: {service_info['Service']}")
    print(f"Service Address (IP): {service_info['Address']}")
    print(f"Service Port: {service_info['Port']}")
    print("-" * 30)

Service ID: /home/stiro/.local/bin/uvic-4ca3
Service Name: /home/stiro/.local/bin/uvic
Service Address (IP): 127.0.1.1
Service Port: 5000
------------------------------


In [28]:
res = c.agent.service.deregister("/home/stiro/.local/bin/uvic-4ca3")
services = c.agent.services()
print(res)


False


### Set a key

In [ ]:
# Set a key
key = 'a'
value = 'b'

c.kv.put(key, value)

print(f"Key '{key}' set successfully!")

### Get a key

In [ ]:
index, data = c.kv.get('a')

if data is not None:
    value = data['Value'].decode('utf-8')  # Must decode from bytes
    print(f"Key value: {value}")
else:
    print("Key not found.")
